In [1]:
import sqlite3

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn
def create_team(conn, team):
    """
    Create a new project into the projects table
    :param conn:
    :param team:
    :return: project id
    """
    sql = ''' INSERT INTO team(team_name,num_tweets,score)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, team)
    conn.commit()
    return cur.lastrowid

    
def create_team(conn, team):
    """
    Create a new project into the projects table
    :param conn:
    :param team:
    :return: project id
    """
    sql = ''' INSERT INTO team(team_name,num_tweets,score)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, team)
    conn.commit()
    return cur.lastrowid


def create_player(conn, player):
    """
    Create a new task
    :param conn:
    :param player:
    :return:
    """

    sql = ''' INSERT INTO player(number,name,team_name,player_id)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, player)
    conn.commit()
    return cur.lastrowid

def create_tweet(conn, tweet):
    """
    Create a new task
    :param conn:
    :param tweet:
    :return:
    """

    sql = ''' INSERT INTO tweet(unique_id,tweet_id,player_id,text,score,magnitude,resultant,created_at)
              VALUES(?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, tweet)
    conn.commit()
    return cur.lastrowid

In [2]:
database = "/home/alex/GreyCroc/tweets/Databases/tweets.db"

conn =  create_connection(database)
#with conn:
    #team = ('Warriors',0,0)
    #team_id = create_team(conn, team)
file = open("Databases/Warriors.csv", "r", encoding="utf8")
content = list(file.read().splitlines())
content.pop(0)
players=[]
for i in content:
    x = i.split(',')[0:3]
    y = ''.join(x)
    y = y.replace(' ', '')
    x.append(y)
    players.append(x)

print(players[0])

['5', 'Kevon Looney', 'Warriors', '5KevonLooneyWarriors']


In [3]:
import sentiments
import tweets

tweeter = tweets.Tweets()
scorer = sentiments.Sentiment_getter()

In [ ]:
'''
CREATE TRIGGER update_player_table_for_deletion
AFTER DELETE ON tweet
BEGIN
UPDATE player
SET
	num_tweets = num_tweets -1,
	score_avg = (score_avg*num_tweets+score_avg-NEW.score)/num_tweets,
	mag_avg = (mag_avg*num_tweets+mag_avg-NEW.magnitude)/num_tweets,
    res_avg = (res_avg*num_tweets+res_avg-NEW.resultant)/num_tweets,
WHERE
	player_id = NEW.player_id;
END;

CREATE TRIGGER update_player_table_for_insertion
AFTER INSERT ON tweet
BEGIN
UPDATE player
SET
	num_tweets = num_tweets +1,
	score_avg = (score_avg*num_tweets-score_avg+NEW.score)/num_tweets,
	mag_avg = (mag_avg*num_tweets-mag_avg+NEW.magnitude)/num_tweets,
    res_avg = (res_avg*num_tweets-res_avg+NEW.resultant)/num_tweets
WHERE
	player_id = NEW.player_id;
END
'''

In [8]:
def set_avg(player_id1):
    sql_query = '''SELECT score,magnitude,resultant FROM tweet WHERE player_id=?'''
    cursor = conn.cursor()
    cursor.execute(sql_query, (player_id1,))
    rows = cursor.fetchall()

    sql_update_query = '''UPDATE player
                    SET
                    score_avg = ?,
                    mag_avg = ?,
                    res_avg = ?
                    WHERE player_id=?'''
    sum_score,sum_mag,sum_res = 0,0,0
    for i in rows:
        sum_score += i[0]
        sum_mag += i[1]
        sum_res += i[2]
    score_avg = sum_score/len(rows)
    mag_avg = sum_mag/len(rows)
    res_avg = sum_res/len(rows)
    cursor.execute(sql_update_query, (score_avg,mag_avg,res_avg,player_id1))
    conn.commit()
    print(score_avg,mag_avg,res_avg)
    return 

for i in players:
    set_avg(i[3])


0.18000000201165675 0.6020000022649765 0.13659999977946286
0.008000001087784767 0.5170000002533197 0.012500000298023181
0.032000002413988114 0.6989999999850989 0.023400002984702607
-0.006756756354022671 0.5162162179681095 -0.008918918802126075
0.07400000087916851 0.6350000023841857 0.0994000011593103
0.009999999552965164 0.5980000029504299 0.036699999767541944
0.12000000201165677 0.47400000438094136 0.0895000011846423
0.19300000384449958 0.6840000033378602 0.16380000399500133
0.11800000168383122 0.5150000040233135 0.0994000012934208
-0.06400000013411045 0.5290000030398369 -0.06599999924749136
0.027000000178813936 0.46900000467896463 0.029800000724196415
-0.02699999913573265 0.873999999538064 -0.03739999934881931
0.013483146100901486 0.36741573296571045 0.0038202248496956087
0.012000000029802322 0.6580000027269125 0.018899997925758326
0.030000000596046447 0.5340000027418137 0.042000000894069654
-0.001234565528086674 0.713580242001716 0.018518516945618013
0.03500000029802322 0.4750000044

In [ ]:
sum_score,sum_mag,sum_res = 0,0,0
for i in r:
    sum_score += i[0]
    sum_mag += i[1]
    sum_res += i[2]
score_avg = sum_score/len(r)
mag_avg = sum_mag/len(r)
res_avg = sum_res/len(r)

In [ ]:
from math import average
average()

In [4]:
#tweet(tweet_id,player_id,score,created_at,text)
'''error_tally = 0
total_success = 0
for player in players:
    create_player(conn, tuple(player))
    tweet_ids,tweet_texts,tweet_dates = tweeter.search_player(player[1])
    for idx,text in enumerate(tweet_texts):
        unique_id = player[3]+str(tweet_ids[idx])
        tweet_id = tweet_ids[idx]
        player_id = player[3]
        #score,mag = scorer.get_sentiment(text)
        resultant = score*mag
        created_at=str(tweet_dates[idx])
        record = (unique_id,tweet_id,player_id,text,score,mag,resultant,created_at)
        with conn:
            try:
                create_tweet(conn,record)
                total_success+=1
                if(total_success%500==0):
                    print('done')
            except:
                error_tally+=1
                continue'''

    

done
done
done


In [ ]:
#for player in players:
    #with conn:
        #player_id = create_player(conn, player)
        #print(player_id)